In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
        ),        
        MessagesPlaceholder(variable_name="chat_history"),	 # 대화 기록용 key
        ("human", "#Question:\n{question}"),  		         # 사용자 질문을 변수로 사용
    ]
)

In [4]:
llm = ChatOpenAI(model_name="gpt-4o-mini")

In [5]:
chain = prompt | llm | StrOutputParser()

In [6]:
# 대화를 기록하는 체인 생성(chain_with_history)

In [8]:
store = {}  # 세션 기록을 저장할 딕셔너리

In [11]:
# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):		            # 예) session_ids가 abc123 
    print(f"[대화 세션ID]: {session_ids}")				
    
    if session_ids not in store:                        # 세션 ID abc123이 store에 없는 경우        
        # 새로운 ChatMessageHistory 객체를 생성하여 store 딕셔너리에 저장
        store[session_ids] = ChatMessageHistory()	        
        
    return store[session_ids]                           # store에 있으면 해당 세션 ID에 대한 세션 기록 반환  

In [12]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  				        # 세션 기록을 가져오는 함수
    input_messages_key="question",  			# 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  		# 기록 메시지의 키
)

In [13]:
chain_with_history.invoke(
    {"question": "나의 이름은 이인환입니다."},		    # 질문 입력
    config={"configurable": {"session_id": "abc123"}},	# 세션 ID 기준으로 대화를 기록합니다.
)

[대화 세션ID]: abc123


'안녕하세요, 이인환님! 어떻게 도와드릴까요?'

In [14]:
chain_with_history.invoke(
    {"question": "내 이름이 뭐라고?"},
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'당신의 이름은 이인환입니다.'